In [ ]:
using CuArrays
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

In [ ]:
batch = 100
learning_rate = 10e-4
sample_int = div(42000, batch)-1

In [ ]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []

for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = convert(Array{Float32}, reshape(train[i, 2:end]./(max), (28,28,1,1)))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end

println("Ready...")

In [ ]:
test = CSV.read("datasets/digit-recognizer/test.csv")
xtest = []
ytest = [nothing for i in 1:size(test)[1]]
for i in 1:size(test)[1]
    max = maximum(test[i,2:end])+0.1
    normalized = convert(Array{Float32}, reshape(test[i, 1:end]./(max), (28,28,1,1)))
    push!(xtest, normalized)
end

println("Ready...")

In [ ]:
mnist_train = Deeplearning.minibatch(xtrain, ytrain, batch, shuffle=true)

In [ ]:
mnist_test = Deeplearning.minibatch(xtest, ytest, batch, shuffle=true)

In [ ]:
function newrelu(x)
    return (x + abs(x))/2
end

In [ ]:
xavierfloat32 = Deeplearning.@float32 xavier

In [ ]:
conv1 = Deeplearning.ConvolutionLayer((3,3,6), (2,2), (1,1), xavierfloat32, newrelu)((28,28,1,1))
conv2 = Deeplearning.ConvolutionLayer((3,3,16), (2,2), (1,1), xavierfloat32, newrelu)(conv1.telemetry.o)
dense1 = Deeplearning.FullyConnectedLayer(120, xavierfloat32, newrelu)(conv2.telemetry.o)
dense2 = Deeplearning.FullyConnectedLayer(84, xavierfloat32, newrelu)(dense1.telemetry.o)
dense3 = Deeplearning.FullyConnectedLayer(10, xavierfloat32, newrelu)(dense2.telemetry.o)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

In [ ]:
function f(x)
    x = conv1(x,s=(2,2))
    x = conv2(x,s=(2,2))    
    x = dense1(x)
    x = dense2(x)
    x = dense3(x)
    x = softmax(x)
end

In [ ]:
lenet5v2 = Deeplearning.Network(layers, f, loss=nll)

In [ ]:
di = collect(Iterators.take(mnist_train,1))

In [ ]:
batch_x, batch_y = di[1]

In [ ]:
cat_batch_x = cat(batch_x...,dims=4)

In [ ]:
using BenchmarkTools

In [ ]:
cubatchx = cu(cat_batch_x)

In [ ]:
@btime f(cubatchx)

In [ ]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

In [ ]:
for (iteration, data) in enumerate(Deeplearning.progress(repeat(mnist_train, 1)))
    batch_x, batch_y = data

    dv = @diff lenet5v2(batch_x, batch_y)

    dparams = @parameters dv

    for pidx in 1:length(dparams[1:end-1])
        dw = grad(dv, dparams[pidx])
        dw .-=learning_rate*dw
    end
    
    if(iteration%sample_int==1)
#         loss_train, loss_test = lenet5v2(mnist_train), lenet5v2(mnist_test)
#         acc_train, acc_test = acc(lenet5v2, mnist_train), acc(lenet5v2, mnist_test)
        loss_train = lenet5v2(mnist_train)
        push!(loss_train_plot, loss_train)
        
        acc_train = acc(lenet5v2, mnist_train)
        push!(acc_train_plot, acc_train)
    end
    
end

In [ ]:
plot([acc_train_plot, acc_test_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_train_plot, loss_test_plot], linewidth=2, title="Loss")